In [10]:
import requests
import json
import io
import gzip

In [2]:
def search_domain(domain):
    record_list = []
    index_list = ["2019-13", "2019-09", "2019-04"]
    print ("[*] Trying target domain: %s" % domain)
    for index in index_list:
        print ("[*] Trying index %s" % index)
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        response = requests.get(cc_url)
        if response.status_code == 200:
            records = response.content.splitlines()
            for record in records:
                record_list.append(json.loads(record))  
            print ("[*] Added %d results." % len(records))
    print ("[*] Found a total of %d hits." % len(record_list))
    return record_list

In [3]:
record_list = search_domain("http://espn.com")

[*] Trying target domain: http://espn.com
[*] Trying index 2019-13
[*] Added 13321 results.
[*] Trying index 2019-09
[*] Added 14320 results.
[*] Trying index 2019-04
[*] Added 14844 results.
[*] Found a total of 42485 hits.


In [11]:
def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'

    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})

    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = io.BytesIO(resp.content+'ÁÇÊ'.encode('utf-8'))
    wrapper = io.TextIOWrapper(raw_data, encoding='utf-8')
    print(wrapper.read())
    
    f = gzip.GzipFile(fileobj=raw_data)

    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""

    if len(data):
        try:
            warc, header, response = data.decode("utf-8").strip().split('\r\n\r\n', 2)
        except:
            pass

    return response

In [12]:
download_page(record_list[1])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [105]:
from newsplease import NewsPlease
article = NewsPlease.from_url('http://www.espn.com/30for30/film?page=animmortalman')
print(article.title)

None


In [104]:
NewsPlease.from_html(page, url=None) 

In [92]:
from nytimesarticle import articleAPI
from bs4 import BeautifulSoup as bs
import urllib.request

In [96]:
api = articleAPI('CKcJc77Nl29KT9UR8lXXnfEQafU2L57E')
articles = api.search(q="Basketball", 
                              fq={#"headline": "Basketball", 
                                  "source": ["The New York Times"]}, 
                              begin_date="20190101", # this can also be an int
                              facet_field=["source", "day_of_week"], 
                              facet_filter=True,
                            page = 1)
articles = articles['response']['docs']

In [101]:
page = urllib.request.urlopen(articles[0]['web_url'])
soup = bs(page.read(), 'html.parser')

In [107]:
soup

<!DOCTYPE html>

<html class="story" itemid="https://www.nytimes.com/2019/04/04/sports/north-carolina-sylvia-hatchell.html" itemscope="true" itemtype="http://schema.org/NewsArticle" lang="en" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<title data-rh="true">North Carolina Women’s Basketball Coach Faces Complaints of Racially Insensitive Comments - The New York Times</title>
<meta content="en-US" data-rh="true" itemprop="inLanguage"/><meta content="2019-04-05T03:50:37.000Z" data-rh="true" itemprop="datePublished" property="article:published"/><meta content="2019-04-06T04:30:06.733Z" data-rh="true" itemprop="dateModified" property="article:modified"/><meta content="en" data-rh="true" http-equiv="Content-Language"/><meta content="noarchive" data-rh="true" name="robots"/><meta content="100000006446275" data-rh="true" itemprop="identifier" name="articleid"/><meta content="nyt://article/52e9e623-2436-598c-922b-8d79c96866cc" data-rh="true" itemprop="identifier" name="nyt_uri"/><me